# 5. Obspy

Obspy is the worldwide library for seismic data manipulation in Python. It includes nearly everything you need to work properly at the institute. 

If you want to search for something into Obspy, the website is really well documented [ObsPy Documentation](http://docs.obspy.org/) and you will find for sure the love of your life on it.

Lesson based on [MESS 2014](https://github.com/obspy/mess2014-notebooks/) and [Seismo-Live](https://krischer.github.io/seismo_live_build/tree/index.html)

The core functionality of ObsPy is provided by..

- ..the most important base classes..
  * the **`UTCDateTime`** class handles time information.
  * the **`Stream`**/**`Trace`** classes handle waveform data.
  * the **`Catalog`**/**`Event`**/... classes handle event metadata (modelled after QuakeML).
  * the **`Inventory`**/**`Station`**/**`Response`**/... classes handle station metadata (modelled after FDSN StationXML).


- ..and the associated functions:
  * The **`read`** function. Reads all kinds of waveform file formats. Outputs a **`Stream`** object.
  * The **`read_events`** function. Reads QuakeML (and MCHEDR) files. Outputs a **`Catalog`** object.
  * The **`read_inventory`** function. Reads FDSN StationXML files. Outputs an **`Inventory`** object.


- the most important classes/functions can be imported from main namespace (`from obspy import ...`)
- Unified interface and functionality for handling waveform data regardless of data source
- **`read`**, **`read_events`** and **`read_inventory`** functions access the appropriate file-format submodule/plugin using filetype autodiscovery
- `obspy.core.util` includes some generally useful utility classes/functions (e.g. for geodetic calculations, Flinn-Engdahl regions, ..)
- some convenience command line scripts are also included (e.g. `obspy-plot`, `obspy-print`, `obspy-scan`, ..)


## 5.2. Station and Event Data

### Station Data

- for station metadata, the de-facto standard of the future (replacing SEED/RESP) is [FDSN StationXML](http://www.fdsn.org/xml/station/)
- FDSN StationXML files can be read using **`read_inventory()`**

In [ ]:
from obspy import read_inventory
# real-world StationXML files often deviate from the official schema definition
# therefore file-format autodiscovery sometimes fails and we have to force the file format
inventory = read_inventory("data/station_PFO.xml", format="STATIONXML")
print(type(inventory))


In [ ]:
inventory

- the nested ObsPy Inventory class structure (Inventory/Station/Channel/Response/...) is closely modelled after FDSN StationXML
<img src="images/Inventory.svg" width=90%>

In [ ]:
print(inventory)

In [ ]:
network = inventory[0]
print(type(network))
print(network)

In [ ]:
station = network[0]
print(type(station))
print(station)

In [ ]:
channel = station[0]
print(type(channel))
print(channel)

In [ ]:
print(channel.response)

In [ ]:
from obspy import read
st = read("data/waveform_PFO.mseed")
print(st)

In [ ]:
inv = read_inventory("data/station_PFO.xml", format="STATIONXML")

In [ ]:
print(st[0].stats)

- the instrument response can be deconvolved from the waveform data using the convenience method **`Stream.remove_response()`**
- evalresp is used internally to calculate the instrument response

In [ ]:
st.plot()
st.remove_response(inventory=inv)
st.plot()

- several options can be used to specify details of the deconvolution (water level, frequency domain prefiltering), output units (velocity/displacement/acceleration), demeaning, tapering and to specify if any response stages should be omitted

In [ ]:
st = read("data/waveform_PFO.mseed")
st.remove_response(inventory=inv, water_level=60, pre_filt=(0.01, 0.02, 8, 10), output="DISP")
st.plot()

- station metadata not present in StationXML yet but in Dataless SEED or RESP files can be used for instrument correction using the `.simulate()` method of Stream/Trace in a similar fashion

### Event Metadata

- for event metadata, the de-facto standard is [QuakeML (an xml document structure)](https://quake.ethz.ch/quakeml/)
- QuakeML files can be read using **`read_events()`**

In [ ]:
from obspy import read_events

catalog = read_events("data/event_tohoku_with_big_aftershocks.xml")
print(catalog)

- **`read_events()`** function returns a **`Catalog`** object, which is
a collection of **`Event`** objects.

In [ ]:
print(type(catalog))
event = catalog[0]
print(type(event))

In [ ]:
event = catalog[0]

print(event.picks)

- Event objects are again collections of other resources.
- the nested ObsPy Event class structure (Catalog/Event/Origin/Magnitude/FocalMechanism/...) is closely modelled after QuakeML
<img src="images/Event.svg" width=90%>

In [ ]:
print(type(event.origins))
print(type(event.origins[0]))
print(event.origins[0])

In [ ]:
print(type(event.magnitudes))
print(type(event.magnitudes[0]))
print(event.magnitudes[0])

In [ ]:
# try event.<Tab> to get an idea what "children" elements event has

- The Catalog object contains some convenience methods to make
working with events easier.
- for example, the included events can be filtered with various keys.

In [ ]:
largest_magnitude_events = catalog.filter("magnitude >= 7.8")
print(largest_magnitude_events)

- There is a basic preview plot using the matplotlib basemap module.

In [ ]:
catalog.plot(projection="local", outfile='data/catalog_plot.png') # cannot be done here

- a (modified) Catalog can be output to file (currently there is write support for QuakeML only)

In [ ]:
largest_magnitude_events.write("data/large_events.xml", format="QUAKEML")

- the event type classes can be used to build up Events/Catalogs/Picks/.. from scratch in custom processing work flows and to share them with other researchers in the de facto standard format QuakeML

In [ ]:
from obspy import UTCDateTime
from obspy.core.event import Catalog, Event, Origin, Magnitude
from obspy.geodetics import FlinnEngdahl

cat = Catalog()
cat.description = "Just a fictitious toy example catalog built from scratch"

e = Event()
e.event_type = "not existing"

o = Origin()
o.time = UTCDateTime(2014, 2, 23, 18, 0, 0)
o.latitude = 47.6
o.longitude = 12.0
o.depth = 10000
o.depth_type = "operator assigned"
o.evaluation_mode = "manual"
o.evaluation_status = "preliminary"
o.region = FlinnEngdahl().get_region(o.longitude, o.latitude)

m = Magnitude()
m.mag = 7.2
m.magnitude_type = "Mw"

m2 = Magnitude()
m2.mag = 7.4
m2.magnitude_type = "Ms"

# also included could be: custom picks, amplitude measurements, station magnitudes,
# focal mechanisms, moment tensors, ...

# make associations, put everything together
cat.append(e)
e.origins = [o]
e.magnitudes = [m, m2]
m.origin_id = o.resource_id
m2.origin_id = o.resource_id

print(cat)
cat.write("data/my_custom_events.xml", format="QUAKEML")
